In [24]:
import numpy as np 
import cv2
from skimage.feature import local_binary_pattern
import time
import pandas as pd

In [25]:
import os
directory_list = list()
dir_path = "full_dataset"
for root, dirs, files in os.walk(dir_path, topdown=False):
    for name in dirs:
        directory_list.append((name))
#print(directory_list)
for directory in directory_list:
    print(directory)

catagory_0
catagory_1
catagory_10
catagory_100
catagory_101
catagory_102
catagory_103
catagory_104
catagory_105
catagory_106
catagory_107
catagory_108
catagory_109
catagory_11
catagory_110
catagory_111
catagory_12
catagory_13
catagory_14
catagory_15
catagory_16
catagory_17
catagory_18
catagory_19
catagory_2
catagory_20
catagory_21
catagory_22
catagory_23
catagory_24
catagory_25
catagory_26
catagory_27
catagory_28
catagory_29
catagory_3
catagory_30
catagory_31
catagory_32
catagory_33
catagory_34
catagory_35
catagory_36
catagory_37
catagory_38
catagory_39
catagory_4
catagory_40
catagory_41
catagory_42
catagory_43
catagory_44
catagory_45
catagory_46
catagory_47
catagory_48
catagory_49
catagory_5
catagory_50
catagory_51
catagory_52
catagory_53
catagory_54
catagory_55
catagory_56
catagory_57
catagory_58
catagory_59
catagory_6
catagory_60
catagory_61
catagory_62
catagory_63
catagory_64
catagory_65
catagory_66
catagory_67
catagory_68
catagory_69
catagory_7
catagory_70
catagory_71
catagory_72


In [26]:
import glob

labels = 'labels.csv'
labelfile = open(labels,'w')
for i in range(len(directory_list)):
    readpath = (dir_path+'/' + directory_list[i]+'/*jpg')
    #print(readpath)
    images = glob.glob(readpath)
    for image in images:
        labelfile.write(image+','+directory_list[i][9:len(directory_list[i])]+'\n')
labelfile.close()

In [27]:
import random
labels = 'labels.csv'
shuffled_labels = 'Shuffled_labels.csv'

labelfile = open(labels, "r")
lines = labelfile.readlines()
labelfile.close()
random.shuffle(lines)

shufflefile = open(shuffled_labels, "w")
shufflefile.writelines(lines)
shufflefile.close()

In [30]:
#MY_LBP
#to check uniform or not 
'''
def uniform(pattern):
    pat= int(pattern)
    a=0
    b=0
    cnt=0
    for i in range(0,8):
        if( i==0 ):
            a= int(pattern/2**(7-i))
        else:
            b= int(pattern/2**(7-i))
            if(b!=a):
                cnt=cnt+1
                a=b
        pattern=pattern%2**(7-i)
    if(cnt<=2):
        return 1 #uniform hbe
    else:
        return 0
items = []
true_items = []
for i in range(0, 256):
    if uniform(i):
        items.append(i)
        true_items.append(i)
items.append(250)
true_items.append(250)
#print(len(items))
def get(image, idx, idy):
    if idx < (len(image)) and idy < len(image[0]) and idx>=0 and idy >=0:
        return 1
    else:
        return 0
    
def dir(x,y,img):
	ax=int(img[x+1,y])-int(img[x,y])
	ay=int(img[x,y-1])-int(img[x,y])
    
	if ax>=0 and ay>=0:
		return 1
	elif ax<0 and ay>=0:
		return 2
	elif ax<0 and ay<0:
		return 3
	elif ax>=0 and ay<0:
		return 4
		
def d_lbp(x,y,img):
    out=[]
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    val = img[x][y]
    for i in range(0, 8):
        n_x = x+fx[i]
        n_y = y+fy[i]
        val1=img[n_x][n_y]
        if val1>=val:
            out.append(1)
        else:
            out.append(0)
    #print(out)
    return out

def add_dict(dic, val):
    if val not in dic.keys():
        dic[250] = dic[250] + 1
    else:
        dic[val] = dic[val] + 1

def lbp(img, dic1):
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    for x in range(1, len(img)-1):
        for y in range(1, len(img[x])-1):
            value = d_lbp(x,y,img)
            val1=0
            for i in range(len(value)):
                if(value[i]==1):
                    val1+=2**(7-i)
            add_dict(dic1, val1)
    return dic1

df = pd.read_csv(shuffled_labels, header=None)
rows = df.iterrows()

X_addrs = []
X_hist = []
Y_hist = []
#row = rows[0]
j = 0

# uncomment below this for local tetra pattern
dic1 = {}


def create_dic():
    for i in range(len(items)):
        dic1[items[i]] = 0

start_time = time.time()   

for row in rows:
    #print(row[1][1])
    create_dic()
    img = cv2.imread(row[1][0])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    new_x = []
    dic1 = lbp(img,dic1)
    x1 = []
    for i in dic1.keys():
        x1.append(dic1[i])
    x1 = np.array(x1)
    #print(x1)
    x1 = x1/np.sum(x1)
    new_x = x1.tolist()
    
    X_hist.append(new_x)
    X_addrs.append(row[1][0])
    Y_hist.append(row[1][1])
    if j%100 == 0:
        elapsed_time = time.time() - start_time
        print(str(j) +" "+str(row[1][0])+"  label = "+str(row[1][1]) + " done in " + str(elapsed_time))
        start_time = time.time()
    j = j + 1


df = pd.DataFrame(X_hist)
df.to_csv("lbp_X_hist.csv",header=None)
df = pd.DataFrame(Y_hist)
df.to_csv("lbp_Y_hist.csv",header=None)
df = pd.DataFrame(X_addrs)
df.to_csv("LBP_address.csv",header=None)

'''
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
X_hist =pd.read_csv('lbp_X_hist.csv', sep=',',header=None)
X_hist = np.array(X_hist)
Y_hist= pd.read_csv('lbp_Y_hist.csv', sep=',',header=None)
Y_hist = np.array(Y_hist)

Y_hist = Y_hist[:,1:]
X_hist = X_hist[:,1:]
X_addrs = pd.read_csv('LBP_address.csv', sep=',',header=None)
X_addrs = X_addrs[:][1]
x = X_hist

In [41]:
#TERNARY PATTERN

#to check uniform or not 
def uniform(pattern):
    pat= int(pattern)
    a=0
    b=0
    cnt=0
    for i in range(0,8):
        if( i==0 ):
            a= int(pattern/2**(7-i))
        else:
            b= int(pattern/2**(7-i))
            if(b!=a):
                cnt=cnt+1
                a=b
        pattern=pattern%2**(7-i)
    if(cnt<=2):
        return 1 #uniform hbe
    else:
        return 0
items = []
true_items = []
for i in range(0, 256):
    if uniform(i):
        items.append(i)
        true_items.append(i)
        
items.append(250)
true_items.append(250)
#print(len(items))

import cv2
import numpy as np
def get(image, idx, idy):
    if idx < (len(image)) and idy < len(image[0]) and idx>=0 and idy >=0:
        return 1
    else:
        return 0
    
def dir(x,y,img):
	ax=int(img[x+1,y])-int(img[x,y])
	ay=int(img[x,y-1])-int(img[x,y])
    
	if ax>=0 and ay>=0:
		return 1
	elif ax<0 and ay>=0:
		return 2
	elif ax<0 and ay<0:
		return 3
	elif ax>=0 and ay<0:
		return 4
		
def d_ternary(x,y,img):
    t = 6
    out=[]
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    val = int(img[x][y])
    for i in range(0, 8):
        n_x = x+fx[i]
        n_y = y+fy[i]
        val1=int(img[n_x][n_y])
        if val1 >= val + t:
            out.append(1)
        elif val1 <= val - t:
            out.append(-1)
        else:
            out.append(0)
    return out

def add_dict(dic, val):
    if val not in dic.keys():
        dic[250] = dic[250] +1
    else:
        dic[val] = dic[val] + 1

def ternary(img, dic1, dic2):
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    for x in range(2, len(img)-2):
        for y in range(2, len(img[x])-2):
            value = d_ternary(x, y, img)
            val1=0
            val2=0
            for i in range(len(value)):
                if(value[i]==1):
                    val1+=2**(7-i)
                elif(value[i]==-1):
                    val2+=2**(7-i)
            add_dict(dic1, val1)
            add_dict(dic2, val2)
    return dic1, dic2


df = pd.read_csv(shuffled_labels, header=None)
rows = df.iterrows()

X_addrs = []
X_hist = []
Y_hist = []
#row = rows[0]
j = 0


# uncomment below this for local tetra pattern
dic1 = {}
dic2 = {}

def create_dic():
    for i in range(len(items)):
        dic1[items[i]] = 0
        dic2[items[i]] = 0

start_time = time.time()      
for row in rows:
    #print(row[1][1])
    create_dic()
    img = cv2.imread(row[1][0])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    new_x = []
    dic1, dic2 = ternary(img,dic1, dic2)
    
    x1 = []
    for i in dic1.keys():
        x1.append(dic1[i])
    x1 = np.array(x1)
    x1 = x1/np.sum(x1)
    
    x2 = []
    for i in dic2.keys():
        x2.append(dic2[i])
    x2 = np.array(x2)
    x2 = x2/np.sum(x2)
    
    
    #new_x = x1.tolist()+x2.tolist()+x3.tolist()+x4.tolist()
    new_x = x1.tolist()+x2.tolist()
    
    X_hist.append(new_x)
    X_addrs.append(row[1][0])
    Y_hist.append(row[1][1])
    if j%100==0:
        elapsed_time = time.time() - start_time
        print(str(j)+" done in " + str(elapsed_time))
        start_time = time.time()
    j = j + 1

df = pd.DataFrame(X_hist)
df.to_csv("Ternary_X_hist.csv",header=None)
df = pd.DataFrame(Y_hist)
df.to_csv("Ternary_Y_hist.csv",header=None)
df = pd.DataFrame(X_addrs)
df.to_csv("Ternary_address.csv",header=None)

X_hist =pd.read_csv('Ternary_X_hist.csv', sep=',',header=None)
X_hist = np.array(X_hist)
Y_hist= pd.read_csv('Ternary_Y_hist.csv', sep=',',header=None)
Y_hist = np.array(Y_hist)

Y_hist = Y_hist[:,1:]
X_hist = X_hist[:,1:]
X_addrs = pd.read_csv('Ternary_address.csv', sep=',',header=None)
X_addrs = X_addrs[:][1]
x = X_hist

0 done in 0.5137042999267578
100 done in 40.21201992034912
200 done in 46.85422205924988
300 done in 49.683610916137695
400 done in 48.69416904449463
500 done in 40.24300479888916
600 done in 36.859976291656494
700 done in 45.243125677108765
800 done in 47.11707401275635
900 done in 52.05625128746033
1000 done in 43.97593116760254
1100 done in 36.22238564491272
1200 done in 40.61286497116089
1300 done in 48.60922145843506
1400 done in 48.2224440574646
1500 done in 48.876068115234375
1600 done in 37.783409118652344
1700 done in 36.070523500442505
1800 done in 48.07850670814514
1900 done in 48.427324056625366
2000 done in 49.53569006919861
2100 done in 43.11645698547363
2200 done in 36.77405762672424
2300 done in 42.836525678634644
2400 done in 47.92861008644104
2500 done in 48.347368001937866
2600 done in 47.90162515640259
2700 done in 36.180408239364624
2800 done in 36.80701470375061
2900 done in 48.35836386680603
3000 done in 47.84565734863281
3100 done in 48.3303804397583
3200 done i

In [43]:
#TETRA_PATTERN 
import cv2
import numpy as np
items = []
true_items = []
def del_dir(dic, direct):
    dic.pop(direct*25000, None)
    for i in true_items:
        dic.pop(direct*1000+i, None)

def get(image, idx, idy):
    if idx < (len(image)) and idy < len(image[0]) and idx>=0 and idy >=0:
        return 1
    else:
        return 0
def dir(x,y,img):
	ax=int(img[x+1,y])-int(img[x,y])
	ay=int(img[x,y-1])-int(img[x,y])
    
	if ax>=0 and ay>=0:
		return 1
	elif ax<0 and ay>=0:
		return 2
	elif ax<0 and ay<0:
		return 3
	elif ax>=0 and ay<0:
		return 4
		
def tetra(x,y,mat):
    out=[]
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    val=mat[x][y]
    for i in range(0, 8):
        a=mat[x+fx[i]][y+fy[i]]
        if a==val:
            out.append(0)
        else:
            out.append(a)
    return out,val

def add_dict(dic,val1,val2,val3,direc):
    #print("val + direc = "+str(val)+"  "+ str(direc))
    d=1
    val=0
    c=1
    for i in range(1,5):
        if(d!=direc):
            if(c==1):
                val=val1
            elif(c==2):
                val=val2
            elif(c==3):
                val=val3
            if (1000*d+val) not in dic.keys():
                dic[25000*d] = dic[25000*d] +1
            else:
                dic[d*1000+val] = dic[d*1000+val] + 1
            c=c+1
        d=d+1
            
def dir_mat(img):
    mat=[]
    n=len(img)
    m=len(img[0])
    for i in range(n):
        mat.append([0] * m)
    
    for x in range(2, len(img)-2):
        for y in range(2, len(img[0])-2):
            mat[x][y]=dir(x,y,img)
    return mat

def local_tetra_pattern(img,dic1, dic2, dic3, dic4,j):
    try:
        mat=dir_mat(img)
        ab=1
        for x in range(2, len(img)-2):
            for y in range(2, len(img[0])-2):
                value, direc = tetra(x,y,mat)
                val1=0
                val2=0
                val3=0
                val4=0
                for i in range(0,len(value)):
                    if(value[i]==1):
                        val1+=2**(7-i)
                    elif(value[i]==2):
                        val2+=2**(7-i)
                    elif(value[i]==3):
                        val3+=2**(7-i)
                    elif(value[i]==4):
                        val4+=2**(7-i)

                if direc == 1:
                    add_dict(dic1, val2,val3,val4,direc)
                elif direc == 2:
                    add_dict(dic2, val1,val3,val4, direc)
                elif direc == 3:
                    add_dict(dic3,val1,val2,val4, direc)
                elif direc == 4:
                    add_dict(dic4, val1,val2,val3, direc)
    except TypeError as e:
        print ("type error "+str(j))
        ab=0
        for i in items:
            dic1[i]=1
            dic2[i]=1
            dic3[i]=1
            dic4[i]=1
        del_dir(dic1,1)
        del_dir(dic2,2)
        del_dir(dic3,3)
        del_dir(dic4,4)
    return  dic1, dic2, dic3, dic4,ab


# In[6]:


#to check uniform or not 
def uniform(pattern):
    pat= int(pattern)
    a=0
    b=0
    cnt=0
    for i in range(0,8):
        if( i==0 ):
            a= int(pattern/2**(7-i))
        else:
            b= int(pattern/2**(7-i))
            if(b!=a):
                cnt=cnt+1
                a=b
        pattern=pattern%2**(7-i)
    if(cnt<=2):
        return 1 #uniform hbe
    else:
        return 0
#items.append(0)
for i in range(0, 256):
    if uniform(i):
        items.append(1*1000+i)
        true_items.append(i)
        items.append(2*1000+i)
        items.append(3*1000+i)
        items.append(4*1000+i)
        #print(i)
items.append(25000)
true_items.append(25000)
items.append(25000*2)
items.append(75000)
items.append(100000)

df = pd.read_csv(shuffled_labels)
rows = df.iterrows()

X_addrs = []
X_hist = []
Y_hist = []
#row = rows[0]
j = 0

# uncomment below this for local tetra pattern
dic1 = {}
dic2 = {}
dic3 = {}
dic4 = {}
dic_m = {}
        
def create_dic():
    for i in range(len(items)):
        dic1[items[i]] = 0
        dic2[items[i]] = 0
        dic3[items[i]] = 0
        dic4[items[i]] = 0
    del_dir(dic1, 1)
    del_dir(dic2, 2)
    del_dir(dic3, 3)
    del_dir(dic4, 4)
    #print((dic1.keys()))
        
        
def magnitude_pattern(img, dic_m,j):
    for i in true_items:
        dic_m[i] = 0
    dic_m[250]=0
    out=[]
    ab=1
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    try:
        for row in range(2, len(img)-2):
            for col in range(2, len(img[0])-2):
                val = 0
                centre = (int(img[row+1][col])-int(img[row][col]))**2 + (int(img[row][col-1])-int(img[row][col]))**2
                for i in range(0, 8):
                    new_row = row+fx[i]
                    new_col = row+fy[i]
                    a = (int(img[new_row+1][new_col])-int(img[new_row][new_col]))**2+(int(img[new_row][new_col-1])-int(img[new_row][new_col]))**2
                    if centre<=a:
                        val+=2**(7-i)
                    if val not in dic_m.keys():
                        dic_m[250] = dic_m[250] +1
                    else:
                        dic_m[val] = dic_m[val] + 1
    except TypeError as e:
        ab=0
        print ("type error magnitude "+str(j))
        for i in true_items:
            dic_m[i] = 1
    return dic_m,ab

start_time = time.time()      
for row in rows:
    #print(row[1][1])
    create_dic()
    img = cv2.imread(row[1][0],0)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #img = cv2.resize(img,(32,32))
    dic1, dic2, dic3, dic4,cnt = local_tetra_pattern(img,dic1, dic2, dic3, dic4,j)
    dic_m,cnt2 = magnitude_pattern(img, dic_m,j)
    
    new_x = []
    
    #change here 
    if cnt==1 and cnt2==1:
        for i in dic1.keys():
            new_x.append(dic1[i])
        for i in dic2.keys():
            new_x.append(dic2[i])
        for i in dic3.keys():
            new_x.append(dic3[i])
        for i in dic4.keys():
            new_x.append(dic4[i])
        for i in dic_m.keys():
            new_x.append(dic_m[i])
        new_x = np.array(new_x)
        hist = new_x/np.sum(new_x)
        #print(hist)
        X_addrs.append(row[1][0])
        X_hist.append(hist)
        Y_hist.append(row[1][1])
        if(j%10 == 0):
            elapsed_time = time.time() - start_time
            start_time = time.time()
            print(str(j)+" done in " + str(elapsed_time))
        j = j + 1
    
df = pd.DataFrame(X_hist)
df.to_csv("Tetra_X_hist.csv",header=None)
df = pd.DataFrame(Y_hist)
df.to_csv("Tetra_Y_hist.csv",header=None)
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
df = pd.DataFrame(X_addrs)
df.to_csv("Tetra_address.csv",header=None)
X_hist =pd.read_csv('Tetra_X_hist.csv', sep=',',header=None)
X_hist = np.array(X_hist)
Y_hist= pd.read_csv('Tetra_Y_hist.csv', sep=',',header=None)
Y_hist = np.array(Y_hist)

Y_hist = Y_hist[:,1:]
X_hist = X_hist[:,1:]
X_addrs = pd.read_csv('Tetra_address.csv', sep=',',header=None)
X_addrs = X_addrs[:][1]
x = X_hist
# total sum = 0.32 0.17 0.1 0.07 0.06 0.04 0.04 0.03 0.02 0.02 

0 done in 2.6824655532836914
10 done in 19.953608751296997
20 done in 20.717150688171387
30 done in 20.211448192596436
40 done in 19.793689727783203
50 done in 17.240150451660156
60 done in 13.587233304977417
70 done in 12.926612138748169
80 done in 13.430324077606201
90 done in 13.536264419555664
100 done in 13.293454647064209
110 done in 13.314420223236084
120 done in 18.21858811378479
130 done in 20.056537866592407
140 done in 20.380354404449463
150 done in 18.84123134613037
160 done in 18.91319251060486
170 done in 20.19545841217041
180 done in 19.86164951324463
190 done in 20.33937931060791
200 done in 21.942458629608154
210 done in 21.58366632461548
220 done in 13.47130012512207
230 done in 13.510297775268555
240 done in 13.846105337142944
250 done in 15.193298816680908
260 done in 14.346800565719604
270 done in 14.09796953201294
280 done in 13.812108993530273
290 done in 17.93477511405945
300 done in 19.727726697921753
310 done in 18.39548683166504
320 done in 19.819673776626587

2630 done in 18.067673444747925
2640 done in 18.466448307037354
2650 done in 18.522414922714233
2660 done in 19.363932609558105
2670 done in 18.461449146270752
2680 done in 18.388492822647095
2690 done in 13.99600338935852
2700 done in 13.140490770339966
2710 done in 13.014561176300049
2720 done in 12.796686172485352
2730 done in 13.625238180160522
2740 done in 13.44734001159668
2750 done in 13.119503498077393
2760 done in 12.799713373184204
2770 done in 14.624640941619873
2780 done in 18.72929835319519
2790 done in 19.253997325897217
2800 done in 20.65319514274597
2810 done in 18.559417009353638
2820 done in 18.63732624053955
2830 done in 18.672329902648926
2840 done in 18.804254055023193
2850 done in 17.929755210876465
2860 done in 18.271557331085205
2870 done in 15.720033645629883
2880 done in 12.307977199554443
2890 done in 12.767702579498291
2900 done in 12.414905786514282
2910 done in 12.34194564819336
2920 done in 13.384350061416626
2930 done in 13.10157322883606
2940 done in 13

5220 done in 19.08409357070923
5230 done in 17.450027465820312
5240 done in 18.528414487838745
5250 done in 19.119070529937744
5260 done in 19.214021921157837
5270 done in 18.363503217697144
5280 done in 17.393079042434692
5290 done in 13.145471811294556
5300 done in 13.369357109069824
5310 done in 12.92261528968811
5320 done in 13.870092630386353
5330 done in 13.065516471862793
5340 done in 13.110507011413574
5350 done in 12.586808919906616
5360 done in 12.685749292373657
5370 done in 18.8572256565094
5380 done in 18.71030306816101
5390 done in 18.354512214660645
5400 done in 18.89521312713623
5410 done in 19.301960229873657
5420 done in 18.759276628494263
5430 done in 19.02212929725647
5440 done in 18.492432117462158
5450 done in 18.52241826057434
5460 done in 18.55239510536194
5470 done in 13.186463832855225
5480 done in 13.310393571853638
5490 done in 13.092518091201782
5500 done in 12.815722465515137
5510 done in 14.23887038230896
5520 done in 13.351369619369507
5530 done in 12.99

7810 done in 16.41062331199646
7820 done in 16.033839464187622
7830 done in 14.395787477493286
7840 done in 11.690303325653076
7850 done in 12.149073839187622
7860 done in 11.559374570846558
7870 done in 11.588428735733032
7880 done in 11.53339171409607
7890 done in 13.561248779296875
7900 done in 16.415618181228638
7910 done in 16.09080481529236
7920 done in 16.411621809005737
7930 done in 16.576528549194336
7940 done in 17.241148233413696
7950 done in 15.870945930480957
7960 done in 16.13676142692566
7970 done in 16.84537124633789
7980 done in 15.880926370620728
7990 done in 15.669044971466064
8000 done in 11.667331218719482
8010 done in 11.745306253433228
8020 done in 11.776270389556885
8030 done in 11.605392932891846
8040 done in 11.574384927749634
8050 done in 11.61636233329773
8060 done in 15.915889024734497
8070 done in 16.46558666229248
8080 done in 16.243736028671265
8090 done in 16.11177349090576
8100 done in 16.473589420318604
8110 done in 16.483580112457275
8120 done in 16.

10380 done in 16.64149045944214
10390 done in 16.53356909751892
10400 done in 16.245697021484375
10410 done in 16.987294673919678
10420 done in 16.676467895507812
10430 done in 14.741576194763184
10440 done in 11.559393405914307
10450 done in 11.95218801498413
10460 done in 11.431467771530151
10470 done in 11.394469261169434
10480 done in 11.645344734191895
10490 done in 11.986149311065674
10500 done in 16.290691375732422
10510 done in 16.08880591392517
10520 done in 15.814963340759277
10530 done in 16.102795839309692
10540 done in 16.652483463287354
10550 done in 16.189748764038086
10560 done in 16.102797269821167
10570 done in 16.592517852783203
10580 done in 16.019845247268677
10590 done in 14.827525615692139
10600 done in 13.761138200759888
10610 done in 11.462465763092041
10620 done in 11.978137254714966
10630 done in 11.47346305847168
10640 done in 11.47342324256897
10650 done in 12.198047399520874
10660 done in 14.203864574432373
10670 done in 16.146774530410767
10680 done in 16

12890 done in 16.36963677406311
12900 done in 16.459622144699097
12910 done in 14.86851191520691
12920 done in 11.561381340026855
12930 done in 12.00916314125061
12940 done in 11.437453985214233
12950 done in 11.53141736984253
12960 done in 11.586384296417236
12970 done in 11.588383674621582
12980 done in 16.09980869293213
12990 done in 16.277708530426025
13000 done in 15.845953464508057
13010 done in 16.0118625164032
13020 done in 16.49158501625061
13030 done in 16.051836490631104
13040 done in 15.864941120147705
13050 done in 15.863945960998535
13060 done in 16.305692195892334
13070 done in 15.99686598777771
13080 done in 13.507306337356567
13090 done in 11.530418157577515
13100 done in 11.885214805603027
13110 done in 11.46543836593628
13120 done in 11.415483236312866
13130 done in 11.753288507461548
13140 done in 13.780134916305542
13150 done in 15.94589877128601
13160 done in 16.105802059173584
13170 done in 15.993873119354248
13180 done in 16.406630277633667
13190 done in 15.8989

15400 done in 14.467734336853027
15410 done in 17.333091735839844
15420 done in 16.62351679801941
15430 done in 16.38661766052246
15440 done in 16.92532730102539
15450 done in 17.067265272140503
15460 done in 16.90132236480713
15470 done in 16.74343204498291
15480 done in 16.985296964645386
15490 done in 16.839377641677856
15500 done in 15.335250854492188
15510 done in 11.79024600982666
15520 done in 12.243018627166748
15530 done in 11.69429898262024
15540 done in 11.963195562362671
15550 done in 11.894245862960815
15560 done in 15.074369668960571
15570 done in 16.862379789352417
15580 done in 16.875340700149536
15590 done in 16.494573831558228
15600 done in 17.16518759727478
15610 done in 16.64049005508423
15620 done in 16.684465885162354
15630 done in 16.794405698776245
15640 done in 16.326666116714478
15650 done in 16.670473337173462
15660 done in 15.728030443191528
15670 done in 11.742314338684082
15680 done in 12.199039936065674
15690 done in 11.732293605804443
15700 done in 11.73

17910 done in 13.756229162216187
17920 done in 12.87074899673462
17930 done in 12.37499189376831
17940 done in 17.166304111480713
17950 done in 18.581506490707397
17960 done in 19.217145204544067
17970 done in 19.24513006210327
17980 done in 19.742846250534058
17990 done in 20.248565196990967
18000 done in 18.26168465614319
18010 done in 18.03581213951111
18020 done in 18.49355435371399
18030 done in 19.086220026016235
18040 done in 15.675146579742432
18050 done in 13.514365673065186
18060 done in 15.50424313545227
18070 done in 16.582634210586548
18080 done in 13.951130151748657
18090 done in 13.927140951156616
18100 done in 13.364434242248535
18110 done in 13.343468189239502
18120 done in 18.98027539253235
18130 done in 17.917877674102783
18140 done in 18.545524835586548
18150 done in 19.81980609893799
18160 done in 19.738849639892578
18170 done in 19.75984025001526
18180 done in 20.16661834716797
18190 done in 19.265111207962036
18200 done in 17.768961668014526
18210 done in 17.7109

20420 done in 16.284704208374023
20430 done in 15.786988019943237
20440 done in 11.599396228790283
20450 done in 11.417463302612305
20460 done in 11.658363819122314
20470 done in 11.49243974685669
20480 done in 11.758267879486084
20490 done in 11.812257289886475
20500 done in 17.11522889137268
20510 done in 16.73245096206665
20520 done in 16.56854224205017
20530 done in 16.83038830757141
20540 done in 16.55555272102356
20550 done in 16.59652304649353
20560 done in 16.46360421180725
20570 done in 17.21418809890747
20580 done in 16.543536901474
20590 done in 16.3406720161438
20600 done in 12.21202802658081
20610 done in 11.580392122268677
20620 done in 11.518425703048706
20630 done in 11.538429737091064
20640 done in 11.520421266555786
20650 done in 11.463456153869629
20660 done in 15.353216886520386
20670 done in 16.645497798919678
20680 done in 16.66050696372986
20690 done in 16.307674407958984
20700 done in 17.052266359329224
20710 done in 16.635504007339478
20720 done in 16.218737363

In [11]:
#AP PATTERN PENTA
items = []
true_items = []
def add_dict(dic,val1,val2,val3):
    val=0
    for i in range(1,4):
        if(i==1):
            val=val1
        elif(i==2):
            val=val2
        elif(i==3):
            val=val3
        if (1000*i+val) not in dic.keys():
            dic[25000*i] = dic[25000*i] +1
        else:
            dic[i*1000+val] = dic[i*1000+val] + 1
    return 1
def add_patrn(dic,val1,val2,val3,val4):
    val=0
    for i in range(4,8):
        if(i==4):
            val=val1
        elif(i==5):
            val=val2
        elif(i==6):
            val=val3
        elif(i==7):
            val=val4
        if(1000*i+val) not in dic.keys():
            dic[25000*i]=dic[25000*i]+1
        else:
            dic[i*1000+val]=dic[i*1000+val]+1
    return 1
def lap(x,y,img):
    t=7
    sums=0
    var=0
    out=[]
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    for i in range(0,8):
        sums+=int(img[x+fx[i]][y+fy[i]])
    mean=sums/8.0
    for i in range(0,8):
        var+=int((mean-int(img[x+fx[i]][y+fy[i]]))**2)
    var=var/8.0
    var=var**(0.5)                                               
    for i in range(0, 8):
        if(int(img[x][y])+t>=int(img[x+fx[i]][y+fy[i]])):
            if(int(img[x][y])-t<=int(img[x+fx[i]][y+fy[i]])):
                out.append(1)
            else:
                out.append(3)
        else:
             out.append(2)
    return out,var,mean
def pattern(var,mean,x,y,img):
    out=[]
    if(var>31):
        p=54
        q=72
        cnt=0
        flag=0
        flag1=1
        cur=0
        prev=0
        first=0
        fx=[0,-1,-1,-1,0,1,1,1,0]
        fy=[1,1,0,-1,-1,-1,0,1,1]
        for i in range(0, 8):
            if(flag==0):
                cur=int(img[x+fx[i]][y+fy[i]])
            if(cur+p<=int(img[x+fx[i+1]][y+fy[i+1]])):
                if(cur+q<=int(img[x+fx[i+1]][y+fy[i+1]])):
                    flag=1
                    out.append(1)
                else:
                    flag=1
                    out.append(3)
            elif(cur-p>=int(img[x+fx[i+1]][y+fy[i+1]])):
                if(cur-q>=int(img[x+fx[i+1]][y+fy[i+1]])):
                    flag=1
                    out.append(2)
                else:
                    flag=1
                    out.append(4)
            else:
                if(flag==0 and flag1):
                    out.append(0)
                elif(flag==1):
                    flag1=0
                    flag=0
                    i=i-1
        return out,1
    else:
        return out,0
def local_ap_pattern(img,dic,q):
    try:
        for x in range(2, len(img)-2):
            for y in range(2, len(img[0])-2):
                value,ab,cd = lap(x,y,img)
                patrn,flag= pattern(ab,cd,x,y,img)
                val1=0
                val2=0
                val3=0
                for i in range(0,len(value)):
                    if(value[i]==1):
                        val1+=2**(7-i)
                    elif(value[i]==2):
                        val2+=2**(7-i)
                    elif(value[i]==3):
                        val3+=2**(7-i)
                add_dict(dic,val1,val2,val3)
                val1=0
                val2=0
                val3=0
                val4=0
                if(flag==1):
                    for i in range(0,len(patrn)):
                        if(value[i]==1):
                            val1+=2**(7-i)
                        elif(value[i]==2):
                            val2+=2**(7-i)
                        elif(value[i]==3):
                            val3+=2**(7-i)
                        elif(value[i]==4):
                            val4+=2**(7-i)
                add_patrn(dic,val1,val2,val3,val4)
    except TypeError as e:
        print ("type error "+str(j))
        return dic,0
    return dic,1
def uniform(pat):
    a=0
    b=0
    cnt=0
    for i in range(0,8):
        if( i==0 ):
            a= int(pat/2**(7-i))
        else:
            b= int(pat/2**(7-i))
            if(b!=a):
                cnt=cnt+1
                a=b
        pat=pat%2**(7-i)
    if(cnt<=2):
        return 1 #uniform hbe
    else:
        return 0
#items.append(0)
for i in range(0, 256):
    if uniform(i):
        items.append(1*1000+i)
        true_items.append(i)
        items.append(2*1000+i)
        items.append(3*1000+i)
        items.append(4*1000+i)
        items.append(5*1000+i)
        items.append(6*1000+i)
        items.append(7*1000+i)
items.append(25000)
items.append(50000)
items.append(25000*3)
items.append(25000*4)
items.append(25000*5)
items.append(25000*6)
items.append(25000*7)
true_items.append(25000)


df = pd.read_csv(shuffled_labels,header=None)
rows = df.iterrows()

X_addrs = []
X_hist = []
Y_hist = []
#row = rows[0]
j = 0
a=-1
# uncomment below this for local tetra pattern
dic = {}
def create_dic():
    #for i in range(len(items)):
    for i in items:
        dic[i] = 0
    return 1
start_time = time.time()      
for row in rows:
    #print(row[1][1])
    create_dic()
    #print(row[1][0])
    img = cv2.imread(row[1][0])
    try:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        dic,a=local_ap_pattern(img,dic,j)
        new_x = []
        if(a==1):
            for i in dic.keys():
                new_x.append(dic[i])
            new_x = np.array(new_x)
            hist = new_x/np.sum(new_x)
            #print(hist)
            X_addrs.append(row[1][0])
            X_hist.append(hist)
            Y_hist.append(row[1][1])
            if(j%500==0):
                elapsed_time = time.time() - start_time
                start_time = time.time()
                print(str(j)+" done in " + str(elapsed_time))
    except Exception as e:
        print ("assertion error "+str(j))
    j = j + 1

df = pd.DataFrame(X_hist)
df.to_csv("AP_Pattern_X_hist.csv",header=None)
df = pd.DataFrame(Y_hist)
df.to_csv("AP_Pattern_Y_hist.csv",header=None)
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
df = pd.DataFrame(X_addrs)
df.to_csv("AP_Pattern_address.csv",header=None)
X_hist =pd.read_csv('AP_Pattern_X_hist.csv', sep=',',header=None)
X_hist = np.array(X_hist)
Y_hist= pd.read_csv('AP_Pattern_Y_hist.csv', sep=',',header=None)
Y_hist = np.array(Y_hist)

Y_hist = Y_hist[:,1:]
X_hist = X_hist[:,1:]
X_addrs = pd.read_csv('AP_Pattern_address.csv', sep=',',header=None)
X_addrs = X_addrs[:][1]
x = X_hist

In [47]:
#AP PATTERN 2 PENTA

import cv2
import numpy as np
items = []
true_items = []
def add_dict(dic,val1,val2,val3):
    val=0
    for i in range(1,4):
        if(i==1):
            val=val1
        elif(i==2):
            val=val2
        elif(i==3):
            val=val3
        if (1000*i+val) not in dic.keys():
            dic[25000*i] = dic[25000*i] +1
        else:
            dic[i*1000+val] = dic[i*1000+val] + 1
    return 1
def add_patrn(dic,val1,val2,val3,val4):
    val=0
    for i in range(4,8):
        if(i==4):
            val=val1
        elif(i==5):
            val=val2
        elif(i==6):
            val=val3
        elif(i==7):
            val=val4
        if(1000*i+val) not in dic.keys():
            dic[25000*i]=dic[25000*i]+1
        else:
            dic[i*1000+val]=dic[i*1000+val]+1
    return 1
def lap(x,y,img):
    t=7
    sums=0
    var=0
    out=[]
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    for i in range(0,8):
        sums+=int(img[x+fx[i]][y+fy[i]])
    mean=sums/8.0
    for i in range(0,8):
        var+=int((mean-int(img[x+fx[i]][y+fy[i]]))**2)
    var=var/8.0
    var=var**(0.5)                                               
    for i in range(0, 8):
        if(int(img[x][y])+t>=int(img[x+fx[i]][y+fy[i]])):
            if(int(img[x][y])-t<=int(img[x+fx[i]][y+fy[i]])):
                out.append(1)
            else:
                out.append(3)
        else:
             out.append(2)
    return out,var,mean
def pattern(var,mean,x,y,img):
    out=[]
    if(var>31):
        p=62
        q=84
        cnt=0
        flag=0
        flag1=1
        cur=0
        prev=0
        first=0
        fx=[0,-1,-1,-1,0,1,1,1,0]
        fy=[1,1,0,-1,-1,-1,0,1,1]
        for i in range(0, 8):
            cur=int(img[x+fx[i]][y+fy[i]])
            if(cur+p<=int(img[x+fx[i+1]][y+fy[i+1]])):
                if(cur+q<=int(img[x+fx[i+1]][y+fy[i+1]])):
                    out.append(1)
                else:
                    out.append(3)
            elif(cur-p>=int(img[x+fx[i+1]][y+fy[i+1]])):
                if(cur-q>=int(img[x+fx[i+1]][y+fy[i+1]])):
                    out.append(2)
                else:
                    out.append(4)
            else:
                    out.append(0)
        return out,1
    else:
        return out,0
def local_ap_pattern(img,dic,q):
    try:
        for x in range(2, len(img)-2):
            for y in range(2, len(img[0])-2):
                value,ab,cd = lap(x,y,img)
                patrn,flag= pattern(ab,cd,x,y,img)
                val1=0
                val2=0
                val3=0
                for i in range(0,len(value)):
                    if(value[i]==1):
                        val1+=2**(7-i)
                    elif(value[i]==2):
                        val2+=2**(7-i)
                    elif(value[i]==3):
                        val3+=2**(7-i)
                add_dict(dic,val1,val2,val3)
                val1=0
                val2=0
                val3=0
                val4=0
                if(flag==1):
                    for i in range(0,len(patrn)):
                        if(value[i]==1):
                            val1+=2**(7-i)
                        elif(value[i]==2):
                            val2+=2**(7-i)
                        elif(value[i]==3):
                            val3+=2**(7-i)
                        elif(value[i]==4):
                            val4+=2**(7-i)
                add_patrn(dic,val1,val2,val3,val4)
    except TypeError as e:
        print ("type error "+str(j))
        return dic,0
    return dic,1
def uniform(pat):
    a=0
    b=0
    cnt=0
    for i in range(0,8):
        if( i==0 ):
            a= int(pat/2**(7-i))
        else:
            b= int(pat/2**(7-i))
            if(b!=a):
                cnt=cnt+1
                a=b
        pat=pat%2**(7-i)
    if(cnt<=2):
        return 1 #uniform hbe
    else:
        return 0
#items.append(0)
for i in range(0, 256):
    if uniform(i):
        items.append(1*1000+i)
        true_items.append(i)
        items.append(2*1000+i)
        items.append(3*1000+i)
        items.append(4*1000+i)
        items.append(5*1000+i)
        items.append(6*1000+i)
        items.append(7*1000+i)
items.append(25000)
items.append(50000)
items.append(25000*3)
items.append(25000*4)
items.append(25000*5)
items.append(25000*6)
items.append(25000*7)
true_items.append(25000)

df = pd.read_csv(shuffled_labels,header=None)
rows = df.iterrows()

X_addrs = []
X_hist = []
Y_hist = []
#row = rows[0]
j = 0
a=-1
# uncomment below this for local tetra pattern
dic = {}
def create_dic():
    #for i in range(len(items)):
    for i in items:
        dic[i] = 0
    return 1
start_time = time.time()      
for row in rows:
    #print(row[1][1])
    create_dic()
    #print(row[1][0])
    img = cv2.imread(row[1][0])
    try:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        dic,a=local_ap_pattern(img,dic,j)
        new_x = []
        if(a==1):
            for i in dic.keys():
                new_x.append(dic[i])
            new_x = np.array(new_x)
            hist = new_x/np.sum(new_x)
            #print(hist)
            X_addrs.append(row[1][0])
            X_hist.append(hist)
            Y_hist.append(row[1][1])
            if(j%500==0):
                elapsed_time = time.time() - start_time
                start_time = time.time()
                print(str(j)+" done in " + str(elapsed_time))
    except Exception as e:
        print ("assertion error "+str(j))
    j = j + 1

df = pd.DataFrame(X_hist)
df.to_csv("AP_Pattern2_X_hist.csv",header=None)
df = pd.DataFrame(Y_hist)
df.to_csv("AP_Pattern2_Y_hist.csv",header=None)
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
df = pd.DataFrame(X_addrs)
df.to_csv("AP_Pattern2_address.csv",header=None)
X_hist =pd.read_csv('AP_Pattern2_X_hist.csv', sep=',',header=None)
X_hist = np.array(X_hist)
Y_hist= pd.read_csv('AP_Pattern2_Y_hist.csv', sep=',',header=None)
Y_hist = np.array(Y_hist)

Y_hist = Y_hist[:,1:]
X_hist = X_hist[:,1:]
X_addrs = pd.read_csv('AP_Pattern2_address.csv', sep=',',header=None)
X_addrs = X_addrs[:][1]
x = X_hist

0 done in 2.293692111968994
500 done in 965.9471564292908
1000 done in 977.4797058105469
1500 done in 962.7465040683746
2000 done in 922.23743891716
2500 done in 919.271178483963
3000 done in 916.5044360160828
3500 done in 909.7906908988953
4000 done in 917.881068944931
4500 done in 930.7608509063721
5000 done in 937.737468957901
5500 done in 847.1720306873322
6000 done in 792.5820615291595
6500 done in 803.5447845458984
7000 done in 831.3541519641876
7500 done in 781.2018849849701
8000 done in 779.201623916626
8500 done in 804.0776782035828
9000 done in 790.8792672157288
9500 done in 791.2577290534973
10000 done in 782.1376874446869
10500 done in 786.5500409603119
11000 done in 787.4265208244324
11500 done in 882.2875444889069
12000 done in 875.8397517204285
12500 done in 899.35822224617
13000 done in 888.3695290088654
13500 done in 876.5623099803925
14000 done in 941.7411019802094
14500 done in 1003.3856067657471
15000 done in 990.8819768428802
15500 done in 955.5531356334686
16000 d

In [45]:
#myproposed_circular (2*59 pattern)
#to check uniform or not 
def uniform(pattern):
    pat= int(pattern)
    a=0
    b=0
    cnt=0
    for i in range(0,8):
        if( i==0 ):
            a= int(pattern/2**(7-i))
        else:
            b= int(pattern/2**(7-i))
            if(b!=a):
                cnt=cnt+1
                a=b
        pattern=pattern%2**(7-i)
    if(cnt<=2):
        return 1 #uniform hbe
    else:
        return 0
items = []
true_items = []
for i in range(0, 256):
    if uniform(i):
        items.append(i)
        true_items.append(i)
        
items.append(250)
true_items.append(250)
#print(len(items))
import cv2
import numpy as np
def get(image, idx, idy):
    if idx < (len(image)) and idy < len(image[0]) and idx>=0 and idy >=0:
        return 1
    else:
        return 0
    
def dir(x,y,img):
	ax=int(img[x+1,y])-int(img[x,y])
	ay=int(img[x,y-1])-int(img[x,y])
    
	if ax>=0 and ay>=0:
		return 1
	elif ax<0 and ay>=0:
		return 2
	elif ax<0 and ay<0:
		return 3
	elif ax>=0 and ay<0:
		return 4
		
def d_lbp(x,y,img):
    t = 0
    out=[]
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    val = int(img[x][y])
    pixels = []
    for i in range(0, 8):
        n_x = x+fx[i]
        n_y = y+fy[i]
        val1=int(img[n_x][n_y])
        pixels.append(val1-val)
        
    for i in range(len(pixels)):
        if pixels[i] >= pixels[(i+1)%len(pixels)] + t:
            out.append(1)
        else:
            out.append(2)
    return out

def add_dict(dic, val):
    if val not in dic.keys():
        dic[250] = dic[250] +1
    else:
        dic[val] = dic[val] + 1

def double_lbp(img, dic1, dic2):
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    for x in range(1, len(img)-1):
        for y in range(1, len(img[x])-1):
            value = d_lbp(x, y, img)
            val1=0
            val2=0
            for i in range(len(value)):
                if(value[i]==1):
                    val1+=2**(7-i)
                elif(value[i]==2):
                    val2+=2**(7-i)
            add_dict(dic1, val1)
            add_dict(dic2, val2)
    return dic1, dic2
df = pd.read_csv(shuffled_labels, header=None)
rows = df.iterrows()

X_addrs = []
X_hist = []
Y_hist = []
#row = rows[0]
j = 0


# uncomment below this for local tetra pattern
dic1 = {}
dic2 = {}
dic3 = {}
dic4 = {}

def create_dic():
    for i in range(len(items)):
        dic1[items[i]] = 0
        dic2[items[i]] = 0
        dic3[items[i]] = 0
        dic4[items[i]] = 0

start_time = time.time()      
for row in rows:
    #print(row[1][1])
    create_dic()
    img = cv2.imread(row[1][0])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    new_x = []
    dic1, dic2 = double_lbp(img,dic1, dic2)
    
    x1 = []
    for i in dic1.keys():
        x1.append(dic1[i])
    x1 = np.array(x1)
    x1 = x1/np.sum(x1)
    
    x2 = []
    for i in dic2.keys():
        x2.append(dic2[i])
    x2 = np.array(x2)
    x2 = x2/np.sum(x2)
    '''
    x3 = []
    for i in dic3.keys():
        x3.append(dic3[i])
    x3 = np.array(x3)
    x3 = x3/np.sum(x3)
    
    x4 = []
    for i in dic4.keys():
        x4.append(dic4[i])
    x4 = np.array(x4)
    x4 = x4/np.sum(x4)
    '''
    #new_x = x1.tolist()+x2.tolist()+x3.tolist()+x4.tolist()
    new_x = x1.tolist()+x2.tolist()
    
    X_hist.append(new_x)
    X_addrs.append(row[1][0])
    Y_hist.append(row[1][1])
    if j%100==0:
        elapsed_time = time.time() - start_time
        print(str(j)+" done in " + str(elapsed_time))
        start_time = time.time()
    j = j + 1
    
df = pd.DataFrame(X_hist)
df.to_csv("Circular_X_hist.csv",header=None)
df = pd.DataFrame(Y_hist)
df.to_csv("Circular_Y_hist.csv",header=None)
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
df = pd.DataFrame(X_addrs)
df.to_csv("Circular_address.csv",header=None)
X_hist =pd.read_csv('Circular_X_hist.csv', sep=',',header=None)
X_hist = np.array(X_hist)
Y_hist= pd.read_csv('Circular_Y_hist.csv', sep=',',header=None)
Y_hist = np.array(Y_hist)

Y_hist = Y_hist[:,1:]
X_hist = X_hist[:,1:]
X_addrs = pd.read_csv('Circular_address.csv', sep=',',header=None)
X_addrs = X_addrs[:][1]
x = X_hist

0 done in 1.0943725109100342
100 done in 86.05992341041565
200 done in 71.18545627593994
300 done in 99.94100642204285
400 done in 91.67870211601257
500 done in 65.41574430465698
600 done in 61.320029497146606
700 done in 89.36104273796082
800 done in 87.65801095962524
900 done in 59.995901584625244
1000 done in 60.81137132644653
1100 done in 83.89815735816956
1200 done in 85.632164478302
1300 done in 60.10682821273804
1400 done in 59.828033208847046
1500 done in 78.28136229515076
1600 done in 78.51322793960571
1700 done in 64.25245809555054
1800 done in 57.060487270355225
1900 done in 72.6615629196167
2000 done in 79.86845445632935
2100 done in 71.6921169757843
2200 done in 56.930567502975464
2300 done in 62.937111139297485
2400 done in 81.0997519493103
2500 done in 75.93669438362122
2600 done in 59.039339780807495
2700 done in 56.91759252548218
2800 done in 77.70168924331665
2900 done in 80.95583462715149
3000 done in 63.99857950210571
3100 done in 56.820738554000854
3200 done in 66.

In [48]:
x = X_hist
def feature_distance(feature1, feature2):
    dist = 0.0
    f1 = np.array(feature1)
    f2 = np.array(feature2)
    dist += np.sum(np.abs(f1-f2))/1+np.sum(f1+f2)
    return dist
f= open("AP2_result.txt","w+")
result = []
sm = [0.0,0.0,0.0, 0.0, 0.0, 0.0,0.0,0.0, 0.0, 0.0]
sm = np.array(sm)
vals = [18, 36, 54, 72, 90, 108, 126, 144, 162, 180]
vals = np.array(vals)
f.write("Label ")
for i in range(10):
    f.write(str((i+1)*10)+"% ")
f.write("\n")
diff = []
for j in range(len(Y_hist)):
    diff.append(x[j])
diff = np.array(diff)        
for i in range(len(Y_hist)):
    if(i%100==0):
        print(i)
    part = 18
    val = 18
    cnt = 1
    true_val = 0
    false_val = 0
    distance_list = []
    query = x[i]
    query_index = i
    q_val = []
    for j in range(len(Y_hist)):
        q_val.append(query)
        #distance_list.append(feature_distance(query, x[j]))
    q_val = np.array(q_val)
    distance_list = np.sum(np.abs(q_val-diff), axis = 1)/(1+np.sum(q_val+diff, axis = 1))
    distance_list = distance_list.tolist()
    #print(distance_list)
    #break
    unsorted = zip(distance_list, Y_hist)
    sorted_touple = sorted(unsorted, key = lambda element : element[0])
    
    recall = []
    head, tail = os.path.split(str(X_addrs[i]))
    while(cnt!=11):
        for j in range(len(distance_list)):
            if(sorted_touple[j][1] == Y_hist[query_index]):
                true_val = true_val + 1
            if(true_val == val):
                recall.append(j)
                cnt+=1
                val = cnt*part
                if(cnt == 11):
                    break
    recall = vals/recall
    sm = sm + recall
    recall = np.round(recall, 2)
    recall = recall.tolist()
    recall = [tail] + recall
    result.append(recall)

sm = sm/len(Y_hist)
sm = np.round(sm, 2)
sorted_list = sorted(result, key = lambda element : element[0])
for j in sorted_list:
    for k in j:
        f.write(str(k)+" ")
    f.write("\n")
f.write("\n total sum = ")
for k in sm:
    f.write(str(k)+" ")
f.write("\n")
f.close() 

'''
query_length =50
true_val = 0
false_val = 0
for i in range(query_length):
    if(sorted_touple[i][1] == Y_hist[query_index]):
        true_val = true_val + 1
    else:
        print("this is wrong "+str(i)+" no image is  confusing with "+str(sorted_touple[i][1]))
        false_val = false_val+1
print(true_val*1.0/query_length*1.0)
'''

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

'\nquery_length =50\ntrue_val = 0\nfalse_val = 0\nfor i in range(query_length):\n    if(sorted_touple[i][1] == Y_hist[query_index]):\n        true_val = true_val + 1\n    else:\n        print("this is wrong "+str(i)+" no image is  confusing with "+str(sorted_touple[i][1]))\n        false_val = false_val+1\nprint(true_val*1.0/query_length*1.0)\n'

In [ ]:
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)

print(X_hist.shape, Y_hist.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_hist), np.array(Y_hist), test_size=0.3)
X_train = np.array(X_train)
print(len(X_train))

'''
for i in range(len(X_train)):
    if(len(X_train[i]) != 26):
       print(len(X_train[i]))
'''


In [ ]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
#clf = svm.SVC()
#clf = GaussianNB()
#clf = LogisticRegression()
#clf = MLPClassifier(alpha=1)
#clf = AdaBoostClassifier()
clf = RandomForestClassifier()
#clf = DecisionTreeClassifier()

#X_train = np.array(X_train)
print(type(X_train))

clf.fit(X_train,Y_train)

clf.score(X_test, Y_test)

In [ ]:
import pickle
from sklearn.model_selection import cross_val_score
#clf = pickle.load(open('RandomForest_model.sav', 'rb'))
clf = RandomForestClassifier()
clf.fit(X_train,Y_train)
scores = cross_val_score(clf,X_test,Y_test,cv=5)
print((scores))
clf.score(X_test, Y_test)

In [ ]:
import pickle
#for saving model
filename = 'DecisionTree_model.sav'
#pickle.dump(clf, open(filename, 'wb'))
clf = pickle.load(open('DecisionTree_model.sav', 'rb'))
clf.score(X_test, Y_test)
#scores = cross_val_score(clf, X_test, Y_test, cv=20)
#print((scores))

In [ ]:
import numpy as np

X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = np.array([1, 1, 2, 2])

from sklearn.svm import SVC

clf = SVC()
print(type(y))
clf.fit(X, y) 
print(clf.predict([[-0.8, -1]]))